# Creating the Image to Dataframe function

This function will take the image inputted and transform the data into a dataframe of rgb, x and y values.

In [3]:
import DISCOVERY
import pandas as pd
def loadImageToDataFrame(fileName): 
  image = DISCOVERY.loadImage(fileName)
  width = len(image)
  height = len(image[0])
  data = []
  for x in range(width):
    for y in range(height):
        pixel = image[x][y]
        r = pixel[0]
        g = pixel[1]
        b = pixel[2]

        d = {"r": r, "g": g, "b": b, "x": x, "y": y}
        data.append(d)

    

  df = pd.DataFrame(data)
  return df

# Creating the Average Image Color function

This function will find the average red, green, and blue values for the image passed in. This function will be vital to find what image will go where in the overal mosaic.



In [4]:
def findAverageImageColor(image):
  sum_r = 0
  sum_g = 0
  sum_b = 0
  count = 0
  for x in range(len(image)):
    for y in range(len(image[0])):
      count = count + 1
      pixel = image[x][y]
      r = pixel[0]
      g = pixel[1]
      b = pixel[2]

      sum_r = sum_r + r
      sum_g = sum_g + g
      sum_b = sum_b + b
      

  avg_r = sum_r / count
  avg_g = sum_g / count
  avg_b = sum_b / count

    
  result = {"avg_r": avg_r, "avg_g": avg_g, "avg_b": avg_b}

 
  return result

# Finding the Average Color of the Tile Images

In this section I found the average color of each tile image and put it in a dataframe.

In [5]:
def createTilesDataFrame(path):
  data = []

  # Loop through all images in the `path` directory:
  for tileImageFileName in DISCOVERY.listTileImagesInPath(path):
    # Load the image as a DataFrame and find the average color:
    image = DISCOVERY.loadImage(tileImageFileName)
    averageColor = findAverageImageColor(image)

    # Store the fileName and average colors in a dictionary:
    d = { "fileName": tileImageFileName, "r": averageColor["avg_r"], "g": averageColor["avg_g"], "b": averageColor["avg_b"] }
    data.append(d)

  # Create the `df_tiles` DataFrame:
  df_tiles = pd.DataFrame(data)
  return df_tiles

# Splitting up and Finding the average color of each tile

This function splits up the base image and finds the average color of each tile. It will be used later to match an image to the tile.

In [6]:
def findAverageImageColorInBox(image, box_x, box_y, box_width, box_height):
  sum_r = 0
  sum_g = 0
  sum_b = 0
  count = 0
  for x in range(box_width):
    for y in range(box_height):
      count = count + 1
      pixel = image[x + box_x][y + box_y]
      r = pixel[0]
      g = pixel[1]
      b = pixel[2]

      sum_r = sum_r + r
      sum_g = sum_g + g
      sum_b = sum_b + b
      

  avg_r = sum_r / count
  avg_g = sum_g / count
  avg_b = sum_b / count

    
  result = {"avg_r": avg_r, "avg_g": avg_g, "avg_b": avg_b}

 
  return result

# Finding the best match

This function findBestTile will take in the tiles dataframe, the average r, g, and b and will use the distance formula to calculate which tiles are the most alike. After using the distance formula on the average r, g and b, it will find the smallest value (the closest ones) and put each tile in its specific place.


In [7]:
# Returns a single row for the title that is the best match given an r_avg, g_avg, and b_avg
import pandas as pd
import numpy

def findBestTile(df_tiles, r_avg, g_avg, b_avg):
  x = df_tiles['r']
  y = df_tiles['g']
  z = df_tiles['b']
  df_tiles["dist"] = numpy.sqrt((x - r_avg)**2 + (y - g_avg)**2 + (z- b_avg)**2)
  return df_tiles.nsmallest(1, "dist")

# Creating the Mosaic

Putting everything together.


In [8]:
# Base image file name
baseImageFileName = "Tiles/basepic.jpeg"

# Folder that contains the file images
tileImageFolder = "Tiles"

# Maximum number of tiles across, more makes a clearer program, should be near 200
maximumTilesX = 101

# Height of tiles
tileHeight = 30

In [9]:
print(f"Creating `df_tiles` from tile images in folder `{tileImageFolder}`...")
df_tiles = createTilesDataFrame(tileImageFolder)
print(f"...found {len(df_tiles)} tile images!")
df_tiles

Creating `df_tiles` from tile images in folder `Tiles`...
...found 101 tile images!


,fileName,r,g,b
0,Tiles/images-30.jpeg,140.585556,110.043333,89.265377
1,Tiles/Unknown-10.jpeg,157.736592,135.788833,118.317556
2,Tiles/1293213332.0.jpg,176.958782,138.669069,132.409518
3,Tiles/images-26.jpeg,141.787560,135.175456,106.895060
4,Tiles/images-3.jpeg,125.087271,111.538252,125.442105
...,...,...,...,...
96,Tiles/images-33.jpeg,141.878274,144.133909,146.378115
97,Tiles/Unknown-13.jpeg,80.735301,78.755787,77.585971
98,Tiles/images-25.jpeg,150.488473,140.998487,147.350553
99,Tiles/Unknown-29.jpeg,147.068274,135.993968,117.615456


In [10]:
import sys
print(f"Loading your base image `{baseImageFileName}`...")
baseImage = DISCOVERY.loadImage(baseImageFileName)
width = len(baseImage)
height = len(baseImage[0])


print(f"Finding best replacement image for each tile...")
# Find the pixelsPerTile to know the pixels used in the base image per mosaic tile:
import math

pixelsPerTile = int(math.ceil(width / maximumTilesX))
width = int(math.floor(width / pixelsPerTile) * pixelsPerTile)
height = int(math.floor(height / pixelsPerTile) * pixelsPerTile)
tilesX = int(width / pixelsPerTile)
tilesY = int(height / pixelsPerTile)

# Create the mosaic:
from PIL import Image
mosaic = Image.new('RGB', (int(tilesX * tileHeight), int(tilesY * tileHeight)))
for x in range(0, width, pixelsPerTile):
  for y in range(0, height, pixelsPerTile):
    avg_color = findAverageImageColorInBox(baseImage, x, y, pixelsPerTile, pixelsPerTile)
    replacement = findBestTile(df_tiles, avg_color["avg_r"], avg_color["avg_g"], avg_color["avg_b"])

    tile = DISCOVERY.getTileImage(replacement["fileName"].values[0], tileHeight)
    mosaic.paste(tile, (int(x / pixelsPerTile) * tileHeight, int(y / pixelsPerTile) * tileHeight))

  # Print out a progress message:
  curRow = int((x / pixelsPerTile) + 1)
  pct = (curRow / tilesX) * 100
  sys.stdout.write(f'\r  ...progress: {curRow * tilesY} / {tilesX * tilesY} ({pct:.2f}%)')

# Save it
mosaic.save('mosaic-hd.jpg')

# Save a smaller one (for posting):
import PIL
d = max(width, height)
factor = d / 4000
if factor <= 1: factor = 1

small_w = width / factor
small_h = height / factor    
baseImage = mosaic.resize( (int(small_w), int(small_h)), resample=PIL.Image.LANCZOS )
baseImage.save('mosaic-web.jpg')

# Print a message:
tada = "\N{PARTY POPPER}"
print("")
print("")
print(f"{tada} MOSAIC COMPLETE! {tada}")
print("- See `mosaic-hq.jpg` to see your HQ moasic! (The file may be HUGE.)")
print("- See `mosaic.jpg` to see a moasic best suited for the web (still big, but not HUGE)!")

Loading your base image `Tiles/basepic.jpeg`...
Finding best replacement image for each tile...
  ...progress: 15200 / 15200 (100.00%)

🎉 MOSAIC COMPLETE! 🎉
- See `mosaic-hq.jpg` to see your HQ moasic! (The file may be HUGE.)
- See `mosaic.jpg` to see a moasic best suited for the web (still big, but not HUGE)!
